In [1]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [2]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [3]:
df = pd.read_sql('SELECT * FROM mcrecos', con=database_connection)
#del df['srno']
del df['index']


In [4]:
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [5]:
df.drop_duplicates(inplace = True)
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [6]:
df['Call_Time_normal'] =  pd.to_datetime(df['Call_Time_normal'], format='%Y-%m-%d')
df['Call_Time_normal'] = df['Call_Time_normal'].dt.date
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05
5211,HDIL,83.0,Rajat Bose,2016-01-04
5212,DLF,125.5,Rajat Bose,2016-01-04
5213,Vinati Organics,555.0,Karvy,2016-01-01


In [9]:
start_date = '2017-01-01'
end_date = '2017-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [10]:
mask = (df['Call_Time_normal'] > date_object1) & (df['Call_Time_normal'] <= date_object2)

# 2017 data

In [11]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [12]:
df1

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Wipro,350.0,ICICI Direct,2017-12-29
1,Petronet LNG,285.0,Kotak Securities,2017-12-28
2,Dishman Carbogen Amcis,435.0,HDFC Securities,2017-12-26
3,Simplex Infrastructure,750.0,ICICI Direct,2017-12-26
4,Greenply Industries,425.0,ICICI Direct,2017-12-26
...,...,...,...,...
1842,GSFC,117.0,ICICI Direct,2017-01-10
1843,Hindalco Industries,176.0,Yogesh Mehta,2017-01-10
1844,Solar Industries,901.0,KR Choksey,2017-01-03
1845,Indian Bank,275.0,ICICI Direct,2017-01-03


In [13]:
print("No of Brokers in 2016 :",df1['Analyst_or_Cmp'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 71


## CLEANING

In [14]:
a = pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'Cmp_Name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sym = sym.replace(",","")
    sym = sym.replace('see23%cagr',"")
    sym = sym.replace("drreddy'slaboratories",'drreddyslabs')
    print(sym)
    sqlrun = "SELECT SYMBOL FROM symbolsmappedmc WHERE recoName = '"+sym+"'"
    a = a.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

wipro
petronetlng
dishmancarbogenamcis
simplexinfrastructure
greenplyindustries
gujaratgas
ksbpumps
bajajelectricals
karurvysyabank
indraprasthagas
teamleaseservices
schandandcompanyltd
atulautoltd
thyrocaretechnologies
techmahindra
havells
bharatbijlee
bhartiairtel
ramasteeltubes
dlf
libasdesignslimited
libasdesigns
dollarindustries
havellsindia
shankarabuildingproducts
dishtv
bharatheavyelectricals
hcltechnologies
icicibank
mcx
l&tfinanceholdings
fiemindustries
infosys
persistentsystems
sadbhavengineering
equitasholdings
ntpc
quesscorp
karnatakabank
alkemlaboratories
tatamotors
pcjeweller
bajajauto
whirlpoolofindia
brigadeenterprises
sterlitetechnologiesltd
futureretail
somanyceramics
statebankofindia
siyaramsilkmills
dilipbuildcon
navkarcorporation
cadilahealthcare
eidparry
jkcement
apollohospitals
suprajitengineering
ntpc
jagranprakashan
gatewaydistriparks
alkemlaboratories
jkcements
amararajabatteries
pfizer
marico
pnbhousingfinance
gujaratnarmadavalley
bankofbaroda
birlacorp
sunp

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'slaboratories'' at line 1
[SQL: SELECT SYMBOL FROM symbolsmappedmc WHERE recoName = 'divi'slaboratories']
(Background on this error at: http://sqlalche.me/e/f405)

In [15]:
len(df1)

1847

In [16]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 1 columns):
SYMBOL    250 non-null object
dtypes: object(1)
memory usage: 2.9+ KB


In [17]:
a["SYMBOL"].fillna(method='ffill', inplace = True) 

In [18]:
a=a.replace("None",np.nan)

In [19]:
a

,SYMBOL
0,WIPRO
1,PETRONET
2,DCAL
3,SIMPLEXINF
4,SIMPLEXINF
...,...
356,GAIL
357,GAIL
358,SHOPERSTOP
359,SHOPERSTOP


## SYMBOLS ADDED TO TABLE

In [20]:
df4 =pd.concat([df1,a],axis=1)

In [21]:
df4 = df4.dropna()

In [22]:
df4

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal,SYMBOL
0,Wipro,350.0,ICICI Direct,2017-12-29,WIPRO
1,Petronet LNG,285.0,Kotak Securities,2017-12-28,PETRONET
2,Dishman Carbogen Amcis,435.0,HDFC Securities,2017-12-26,DCAL
3,Simplex Infrastructure,750.0,ICICI Direct,2017-12-26,SIMPLEXINF
4,Greenply Industries,425.0,ICICI Direct,2017-12-26,SIMPLEXINF
...,...,...,...,...,...
356,GNA Axles,376.0,Centrum,2017-10-06,GAIL
357,CARE Ratings,1722.0,Edelweiss,2017-10-06,GAIL
358,Natco Pharma,1065.0,ICICI Direct,2017-10-05,SHOPERSTOP
359,Bata India,880.0,ICICI Direct,2017-10-04,SHOPERSTOP


In [23]:
uniqueBrokers = df4.Analyst_or_Cmp.unique()
uniqueBrokers

array(['ICICI Direct', 'Kotak Securities', 'HDFC Securities',
       'KR Choksey', 'Motilal Oswal', 'Nirmal Bang', 'Mehta Equites',
       'Edelweiss', 'Dalmia Securities', 'Khambatta Securities',
       'Axis Direct', 'JM Financial', 'Centrum', 'GEPL Capital',
       'Cholamandalam Securities', 'Bonanza', 'Joindre Capital', 'Geojit',
       'Joindre Capital Services', 'Dolat Capital', 'Shahina Mukadam',
       'Centrum Research', 'Cholamandalam securities', 'SPA Securities',
       'Sharekhan', 'Reliance Securities', 'Prabhudas Lilladher',
       'Share India', 'Geojit Research', 'KRChoksey', 'LKP Research',
       'SPA Research', 'BP Equities'], dtype=object)

In [24]:
#uniqueBrokers = uniqueBrokers[:1]

## PERFORMANCE DEDUCTION ALGORITHM

In [25]:
#del brokerPerformance1
brokerPerformance1 = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos


In [26]:
lost =[]
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['Analyst_or_Cmp'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(sym)

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['Call_Time_normal']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
            print(mydate1)
            
            #Fetching current price from nse bav copy
            cnx = mysql.connector.connect(user="root", password='yash', host="127.0.0.1", port=3306, database='mainschema')
            sql_select_Query = "SELECT Closeval FROM "+dbyear1+" WHERE Date = '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            cursor = cnx.cursor()
            cursor.execute(sql_select_Query)
            record = cursor.fetchall()
            if len(record) != 0:
                
                x = int(record[0][0])
            
        
                sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
                dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
                del dfprices1['index']
        
                sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
                dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
                del dfprices2['index']
                dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

                targetp = float(row['Target_Price'])
        
                print("SYMBOL : ",sym)
                dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
                total = total + 1
            
                if not dfhits.empty :
                    print("HIT!")
                    hits = hits + 1
                    #recbuying = float(row['recomended_buying'])
                    maxval = dfprices['Closeval'].max()
                    growthperc = (maxval - x) *100 / x
                    print('Growth : ',growthperc)
                    growthSum = growthSum + growthperc
                    
                
                else:
                    print("DIDN'T HIT!")
                del dfprices
                del dfhits
            else:
                lost.append(sym)
    if total != 0: 
        hitRatio = int(hits*100 / total)
        print("Growthsum",growthSum)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformance1 = brokerPerformance1.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  ICICI Direct
WIPRO
2017-12-29


C:\Users\dhruv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



SYMBOL :  WIPRO
DIDN'T HIT!
SIMPLEXINF
2017-12-26
SYMBOL :  SIMPLEXINF
DIDN'T HIT!
SIMPLEXINF
2017-12-26
SYMBOL :  SIMPLEXINF
HIT!
Growth :  8.696027633851472
GUJGASLTD
2017-12-26
SYMBOL :  GUJGASLTD
DIDN'T HIT!
GUJGASLTD
2017-12-26
SYMBOL :  GUJGASLTD
DIDN'T HIT!
BAJAJELEC
2017-12-26
SYMBOL :  BAJAJELEC
HIT!
Growth :  44.18604651162791
SHANKARA
2017-12-06
SYMBOL :  SHANKARA
DIDN'T HIT!
DISHTV
2017-12-06
SYMBOL :  DISHTV
DIDN'T HIT!
ALKEM
2017-11-28
SYMBOL :  ALKEM
HIT!
Growth :  17.6506329113924
SOMANYCERA
2017-11-28
SYMBOL :  SOMANYCERA
HIT!
Growth :  10.954598370197907
SIYSIL
2017-11-28
SYMBOL :  SIYSIL
HIT!
Growth :  19.945736434108525
RAMCOCEM
2017-11-15
SYMBOL :  RAMCOCEM
HIT!
Growth :  22.992857142857147
CIPLA
2017-11-15
SYMBOL :  CIPLA
HIT!
Growth :  13.456998313659351
DBCORP
2017-11-15
SYMBOL :  DBCORP
DIDN'T HIT!
DBCORP
2017-11-15
SYMBOL :  DBCORP
HIT!
Growth :  5.4005524861878484
DBCORP
2017-11-15
SYMBOL :  DBCORP
DIDN'T HIT!
TECHM
2017-11-15
SYMBOL :  TECHM
HIT!
Growth :  5

SYMBOL :  KOTAKBANK
HIT!
Growth :  36.87560738581148
ICICIPRULI
2017-10-27
SYMBOL :  ICICIPRULI
DIDN'T HIT!
ZEEL
2017-10-26
SYMBOL :  ZEEL
HIT!
Growth :  15.517890772128057
KOTAKBANK
2017-10-26
SYMBOL :  KOTAKBANK
HIT!
Growth :  38.35461689587427
IBULHSGFIN
2017-10-23
SYMBOL :  IBULHSGFIN
DIDN'T HIT!
BLUEDART
2017-10-23
SYMBOL :  BLUEDART
HIT!
Growth :  16.54089581304771
BAJFINANCE
2017-10-18
SYMBOL :  BAJFINANCE
HIT!
Growth :  59.823340471092074
FEDERALBNK
2017-10-18
SYMBOL :  FEDERALBNK
DIDN'T HIT!
DELTACORP
2017-10-16
SYMBOL :  DELTACORP
DIDN'T HIT!
INDUSINDBK
2017-10-13
SYMBOL :  INDUSINDBK
HIT!
Growth :  15.602857142857141
INDUSINDBK
2017-10-13
SYMBOL :  INDUSINDBK
HIT!
Growth :  15.602857142857141
BHARTIARTL
2017-10-13
SYMBOL :  BHARTIARTL
DIDN'T HIT!
CYIENT
2017-10-12
SYMBOL :  CYIENT
HIT!
Growth :  70.36821705426357
CYIENT
2017-10-12
SYMBOL :  CYIENT
DIDN'T HIT!
NATCOPHARM
2017-10-12
SYMBOL :  NATCOPHARM
DIDN'T HIT!
AJANTPHARM
2017-10-12
SYMBOL :  AJANTPHARM
HIT!
Growth :  37.5

SYMBOL :  BAJFINANCE
HIT!
Growth :  53.1025641025641
BAJFINANCE
2017-10-10
SYMBOL :  BAJFINANCE
HIT!
Growth :  53.1025641025641
CENTURYPLY
2017-10-09
SYMBOL :  CENTURYPLY
DIDN'T HIT!
COCHINSHIP
2017-10-09
SYMBOL :  COCHINSHIP
HIT!
Growth :  12.344961240310086
COCHINSHIP
2017-10-09
SYMBOL :  COCHINSHIP
DIDN'T HIT!
Growthsum 462.9275144118348
Broker :  Axis Direct HitRatio :  55 Avg. Growth :  9 Total Recos. :  47
brokerName :  JM Financial
BHEL
2017-12-06
SYMBOL :  BHEL
DIDN'T HIT!
ICICIBANK
2017-12-06
SYMBOL :  ICICIBANK
HIT!
Growth :  23.745819397993312
L&TFH
2017-12-05
SYMBOL :  L&TFH
DIDN'T HIT!
DBL
2017-11-21
SYMBOL :  DBL
DIDN'T HIT!
ORIENTREF
2017-11-21
SYMBOL :  ORIENTREF
DIDN'T HIT!
ORIENTREF
2017-11-21
SYMBOL :  ORIENTREF
DIDN'T HIT!
ORIENTREF
2017-11-21
SYMBOL :  ORIENTREF
HIT!
Growth :  72.1875
ENIL
2017-11-13
SYMBOL :  ENIL
HIT!
Growth :  1.6666666666666725
DRREDDY
2017-11-13
SYMBOL :  DRREDDY
HIT!
Growth :  12.308020477815699
DRREDDY
2017-11-13
SYMBOL :  DRREDDY
HIT!
Growt

SYMBOL :  MARUTI
HIT!
Growth :  19.54346504559271
Growthsum 19.54346504559271
Broker :  Share India HitRatio :  100 Avg. Growth :  19 Total Recos. :  1
brokerName :  Geojit Research
IOC
2017-11-02
SYMBOL :  IOC
HIT!
Growth :  0.9420289855072409
ICICIBANK
2017-11-02
SYMBOL :  ICICIBANK
DIDN'T HIT!
INDUSINDBK
2017-10-11
SYMBOL :  INDUSINDBK
DIDN'T HIT!
Growthsum 0.9420289855072409
Broker :  Geojit Research HitRatio :  33 Avg. Growth :  0 Total Recos. :  3
brokerName :  KRChoksey
CROMPTON
2017-10-31
SYMBOL :  CROMPTON
DIDN'T HIT!
CROMPTON
2017-10-31
SYMBOL :  CROMPTON
DIDN'T HIT!
ICICIBANK
2017-10-31
SYMBOL :  ICICIBANK
HIT!
Growth :  20.76666666666667
CUMMINSIND
2017-10-31
SYMBOL :  CUMMINSIND
HIT!
Growth :  7.508287292817685
Growthsum 28.274953959484353
Broker :  KRChoksey HitRatio :  50 Avg. Growth :  7 Total Recos. :  4
brokerName :  LKP Research
SHOPERSTOP
2017-10-30
SYMBOL :  SHOPERSTOP
HIT!
Growth :  22.30555555555556
CANFINHOME
2017-10-09
SYMBOL :  CANFINHOME
HIT!
Growth :  0.0262

In [27]:
len(lost)

4

In [29]:
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos
0,ICICI Direct,54,13,48
1,Kotak Securities,20,2,10
2,HDFC Securities,45,9,20
3,KR Choksey,46,12,15
4,Motilal Oswal,53,30,49
5,Nirmal Bang,0,0,1
6,Edelweiss,51,14,54
7,Dalmia Securities,100,49,2
8,Khambatta Securities,66,12,6
9,Axis Direct,55,9,47


In [30]:
brokerPerformance1.to_sql(con=database_connection, name='MCbrokerperformance2017', if_exists='replace')

C:\Users\dhruv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'MCbrokerperformance2017' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
